In [1]:
# CNT Cosmological Drift Test — Single Mega-Cell
# ------------------------------------------------
# Simulates a high-density cognitive field Ψ on a 2D torus and shows:
#  • Cooling:        T(t) ∝ Var[Ψ]  decreases
#  • Expansion:      a(t) = ξ(t)/ξ(0) increases (ξ = correlation length)
#  • Curvature drop: Θ(t) = ⟨|∇²Ψ|⟩ decreases
#  • CNT invariant:  Ω(t) = Φ/Θ with Φ = ⟨Ψ⟩² / Var[Ψ]
#
# Outputs saved to OUTDIR:
#  - metrics_timeseries.csv
#  - psi_snapshot_t0.png, psi_snapshot_t{mid}.png, psi_snapshot_t{end}.png
#  - temperature_vs_time.png
#  - scale_factor_vs_time.png
#  - curvature_vs_time.png
#  - omega_vs_time.png
#  - loglog_T_vs_a.png (slope ~ -1 is CMB-like cooling law)
#
# Charting: pure matplotlib, one figure per chart, no custom colors.

import os, sys, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# -----------------------------
# Tweakables
# -----------------------------
# Save location (edit if you want a different folder on Windows)
OUTDIR = os.path.abspath("./cnt_cosmo_drift")

# Grid & runtime
N      = 256      # grid size (N x N)
DX     = 1.0      # lattice spacing
DT     = 0.1      # time step
STEPS  = 1200     # total iterations

# Dynamics
D        = 0.6    # diffusion coefficient
LAMBDA   = 0.02   # cubic relaxation (prevents trivial exponential flattening)
NOISE_AMP = 0.0   # optional tiny noise (0 for clarity)

# Snapshots
SAVE_SNAPSHOTS  = True
SNAPSHOT_STEPS  = [0, STEPS//2, STEPS-1]

# Reproducibility
np.random.seed(7)

# -----------------------------
# Utilities
# -----------------------------
def laplacian_periodic(X, dx):
    return (
        -4.0 * X
        + np.roll(X, 1, 0) + np.roll(X, -1, 0)
        + np.roll(X, 1, 1) + np.roll(X, -1, 1)
    ) / (dx * dx)

def structure_factor(Psi):
    F = np.fft.fftn(Psi)
    S = (F * np.conj(F)).real
    return np.fft.fftshift(S)

def radial_average(S2d):
    ny, nx = S2d.shape
    cy, cx = ny//2, nx//2
    y, x = np.indices((ny, nx))
    r = np.sqrt((y - cy)**2 + (x - cx)**2).astype(np.int32)
    rmax = min(cy, cx)
    sums   = np.bincount(r.ravel(), weights=S2d.ravel(), minlength=rmax)
    counts = np.bincount(r.ravel(), minlength=rmax)
    with np.errstate(divide='ignore', invalid='ignore'):
        radial = np.where(counts > 0, sums / counts, 0.0)
    return radial  # index ~ pixel radius

def estimate_correlation_length(Psi, dx):
    # ξ from structure factor half-height method on radial S(k)
    S2d  = structure_factor(Psi)
    Srad = radial_average(S2d)
    if len(Srad) < 3:
        return 1.0
    S0 = Srad[0]
    if S0 <= 0:
        return 1.0
    half = 0.5 * S0
    idx = None
    for i in range(1, len(Srad)):
        if Srad[i] <= half:
            idx = i
            break
    if idx is None:
        idx = len(Srad) - 1
    # pixel radius -> |k| ≈ 2π * r / (N*dx)
    k_mag = 2.0 * np.pi * idx / (N * dx)
    if k_mag <= 1e-8:
        return float(N * dx)
    return float(1.0 / k_mag)

def save_field_image(Psi, path, title):
    plt.figure(figsize=(6, 6))
    im = plt.imshow(Psi, origin='lower', interpolation='nearest')
    plt.title(title)
    plt.colorbar(im, shrink=0.8)
    plt.tight_layout()
    plt.savefig(path, dpi=160)
    plt.close()

# -----------------------------
# Initialize & Output dir
# -----------------------------
os.makedirs(OUTDIR, exist_ok=True)

# Start with compressed/high-curvature Ψ (emphasize high-k)
Psi = np.random.normal(0, 1.0, size=(N, N))
F   = np.fft.fftn(Psi)
ky  = np.fft.fftfreq(N, d=DX)
kx  = np.fft.fftfreq(N, d=DX)
KX, KY = np.meshgrid(kx, ky, indexing='xy')
K2  = KX**2 + KY**2
filter_mask = np.exp(- (K2) / (0.15 * (np.max(K2) + 1e-12)))  # suppress low-k
F *= (1.0 - filter_mask)
Psi = np.fft.ifftn(F).real
Psi *= 2.5

# -----------------------------
# Metrics
# -----------------------------
ts, temps, a_list, curv, coh, omega = [], [], [], [], [], []

xi0 = estimate_correlation_length(Psi, DX)

if SAVE_SNAPSHOTS and 0 in SNAPSHOT_STEPS:
    save_field_image(Psi, os.path.join(OUTDIR, "psi_snapshot_t0.png"), "Ψ at t=0")

# -----------------------------
# Time evolution
# -----------------------------
for step in range(STEPS):
    Lap     = laplacian_periodic(Psi, DX)
    Theta_t = float(np.mean(np.abs(Lap)))            # curvature proxy
    Psi     = Psi + DT * (D * Lap - LAMBDA * Psi**3) # diffusion + cubic relax
    if NOISE_AMP > 0:
        Psi += np.random.normal(0, NOISE_AMP, size=Psi.shape) * math.sqrt(DT)

    var   = float(np.var(Psi))
    mean  = float(np.mean(Psi))
    T_t   = var                                   # temperature proxy
    xi_t  = estimate_correlation_length(Psi, DX)  # correlation length
    a_t   = float(xi_t / (xi0 + 1e-12))           # scale factor
    Phi_t = (mean * mean) / (var + 1e-12)         # coherence proxy
    Om_t  = Phi_t / (Theta_t + 1e-12)

    t_now = step * DT
    ts.append(t_now); temps.append(T_t); a_list.append(a_t)
    curv.append(Theta_t); coh.append(Phi_t); omega.append(Om_t)

    if SAVE_SNAPSHOTS and step in SNAPSHOT_STEPS:
        save_field_image(Psi, os.path.join(OUTDIR, f"psi_snapshot_t{step}.png"),
                         f"Ψ at t={t_now:.1f}")

# Ensure final snapshot exists
if SAVE_SNAPSHOTS and (STEPS-1) not in SNAPSHOT_STEPS:
    save_field_image(Psi, os.path.join(OUTDIR, f"psi_snapshot_t{STEPS-1}.png"),
                     f"Ψ at t={(STEPS-1)*DT:.1f}")

# -----------------------------
# Save metrics
# -----------------------------
df = pd.DataFrame({
    "t": ts,
    "T": temps,
    "a": a_list,
    "Theta": curv,
    "Phi": coh,
    "Omega": omega
})
csv_path = os.path.join(OUTDIR, "metrics_timeseries.csv")
df.to_csv(csv_path, index=False)

# -----------------------------
# Plots (one per figure)
# -----------------------------
plt.figure(figsize=(7,4))
plt.plot(df["t"], df["T"], linewidth=2)
plt.xlabel("time"); plt.ylabel("T (∝ Var[Ψ])"); plt.title("Cooling: Temperature T(t)")
plt.tight_layout(); plt.savefig(os.path.join(OUTDIR, "temperature_vs_time.png"), dpi=160); plt.close()

plt.figure(figsize=(7,4))
plt.plot(df["t"], df["a"], linewidth=2)
plt.xlabel("time"); plt.ylabel("a(t) = ξ(t)/ξ(0)"); plt.title("Expansion: Effective Scale Factor a(t)")
plt.tight_layout(); plt.savefig(os.path.join(OUTDIR, "scale_factor_vs_time.png"), dpi=160); plt.close()

plt.figure(figsize=(7,4))
plt.plot(df["t"], df["Theta"], linewidth=2)
plt.xlabel("time"); plt.ylabel("Θ(t) = ⟨|∇²Ψ|⟩"); plt.title("Curvature Reduction Θ(t)")
plt.tight_layout(); plt.savefig(os.path.join(OUTDIR, "curvature_vs_time.png"), dpi=160); plt.close()

plt.figure(figsize=(7,4))
plt.plot(df["t"], df["Omega"], linewidth=2)
plt.xlabel("time"); plt.ylabel("Ω(t) = Φ/Θ"); plt.title("CNT Invariant Ω(t)")
plt.tight_layout(); plt.savefig(os.path.join(OUTDIR, "omega_vs_time.png"), dpi=160); plt.close()

# -----------------------------
# Optional: log–log T vs a (fit slope ~ -1 is CMB-like)
# -----------------------------
aa = np.array(a_list); TT = np.array(temps)
mask = (aa > 0) & (TT > 0)
loga = np.log(aa[mask]); logT = np.log(TT[mask])
if loga.size >= 10:
    slope, intercept = np.polyfit(loga, logT, 1)
else:
    slope, intercept = float('nan'), float('nan')

plt.figure(figsize=(6,5))
plt.scatter(loga, logT, s=10)
xline = np.linspace(loga.min(), loga.max(), 200)
plt.plot(xline, slope*xline + intercept, linewidth=2)
plt.xlabel("log a"); plt.ylabel("log T"); plt.title(f"log T vs log a (slope ≈ {slope:.3f})")
plt.tight_layout(); plt.savefig(os.path.join(OUTDIR, "loglog_T_vs_a.png"), dpi=160); plt.close()

print(f"Saved artifacts to: {OUTDIR}")
for f in sorted(os.listdir(OUTDIR)):
    print(" -", f)


Saved artifacts to: C:\Users\caleb\CNT_Lab\notebooks\archive\cnt_cosmo_drift
 - curvature_vs_time.png
 - loglog_T_vs_a.png
 - metrics_timeseries.csv
 - omega_vs_time.png
 - psi_snapshot_t0.png
 - psi_snapshot_t1199.png
 - psi_snapshot_t600.png
 - scale_factor_vs_time.png
 - temperature_vs_time.png


In [2]:
# CNT Cosmological Drift Test — v2 (smoother ξ, better T, robust slope)
import os, math, numpy as np, pandas as pd, matplotlib.pyplot as plt
np.random.seed(7)

# ---------------- Tweakables ----------------
OUTDIR = os.path.abspath("./cnt_cosmo_drift_v2")
N, DX, DT, STEPS = 256, 1.0, 0.1, 1500
D, LAMBDA, NOISE_AMP = 0.28, 0.002, 0.0
SAVE_SNAPSHOTS = True
SNAPSHOT_STEPS = [0, STEPS//2, STEPS-1]

os.makedirs(OUTDIR, exist_ok=True)

# -------------- Helpers --------------------
def laplacian_periodic(X, dx):
    return (-4*X + np.roll(X,1,0)+np.roll(X,-1,0)+np.roll(X,1,1)+np.roll(X,-1,1))/(dx*dx)

def grad_energy(Psi, dx):
    gx = (np.roll(Psi,-1,1) - np.roll(Psi,1,1))/(2*dx)
    gy = (np.roll(Psi,-1,0) - np.roll(Psi,1,0))/(2*dx)
    return float(np.mean(gx*gx + gy*gy))

def corr_length_from_C(Psi, max_r=None):
    # Isotropic two-point C(r) = <Psi(x) Psi(x+r)>_angles, then fit C(r) ~ C0 * exp(-r/xi)
    # Efficient via FFT: corr = ifft |F|^2, then radial average.
    F = np.fft.fftn(Psi)
    S = (F*np.conj(F)).real
    corr2d = np.fft.ifftn(S).real / (Psi.size)  # normalized autocorr
    corr2d = np.fft.fftshift(corr2d)
    ny, nx = corr2d.shape
    cy, cx = ny//2, nx//2
    y, x = np.indices((ny, nx))
    r = np.sqrt((y-cy)**2 + (x-cx)**2)
    r_i = r.astype(np.int32)
    rmax = min(cy, cx) if max_r is None else min(max_r, min(cy, cx))
    sums = np.bincount(r_i.ravel(), weights=corr2d.ravel(), minlength=rmax)
    cnts = np.bincount(r_i.ravel(), minlength=rmax)
    Cr = np.where(cnts>0, sums/cnts, 0.0)[:rmax]
    # Normalize to C(0)=1 for stable fitting
    if Cr[0] <= 0: return 1.0
    Cr = Cr/Cr[0]
    rr = np.arange(len(Cr))*DX
    # Fit log(C) ~ -r/xi over a central window (avoid r=0 and far tail)
    i0 = max(2, len(rr)//200)           # skip first couple of bins
    i1 = max(i0+10, len(rr)//3)         # use first third for fit
    y = Cr[i0:i1]
    x = rr[i0:i1]
    y = np.clip(y, 1e-12, 1)            # guard
    slope, intercept = np.polyfit(x, np.log(y), 1)
    xi = -1.0/slope if slope < 0 else 1.0
    return float(xi)

def save_field(Psi, path, title):
    plt.figure(figsize=(6,6))
    im = plt.imshow(Psi, origin='lower', interpolation='nearest')
    plt.title(title); plt.colorbar(im, shrink=0.8)
    plt.tight_layout(); plt.savefig(path, dpi=160); plt.close()

# -------------- Initialize Ψ ----------------
Psi = np.random.normal(0,1.0,(N,N))
# emphasize high-k slightly, but less aggressively than v1
F = np.fft.fftn(Psi)
ky = np.fft.fftfreq(N, d=DX); kx = np.fft.fftfreq(N, d=DX)
KX, KY = np.meshgrid(kx, ky, indexing='xy'); K2 = KX**2 + KY**2
mask = np.exp(-K2/(0.08*(K2.max()+1e-12)))
F *= (1.0 - 0.7*mask)
Psi = np.fft.ifftn(F).real
Psi *= 1.6

# -------------- Evolve ----------------------
ts, T_list, a_list, Theta_list, Phi_list, Om_list = [], [], [], [], [], []
xi0 = corr_length_from_C(Psi)

if SAVE_SNAPSHOTS and 0 in SNAPSHOT_STEPS:
    save_field(Psi, os.path.join(OUTDIR,"psi_t0.png"), "Ψ at t=0")

for step in range(STEPS):
    Lap = laplacian_periodic(Psi, DX)
    Theta_t = float(np.mean(np.abs(Lap)))
    Psi = Psi + DT*(D*Lap - LAMBDA*Psi**3)
    if NOISE_AMP>0:
        Psi += np.random.normal(0, NOISE_AMP, size=Psi.shape)*math.sqrt(DT)

    # Metrics
    var  = float(np.var(Psi))
    mean = float(np.mean(Psi))
    T_t  = grad_energy(Psi, DX)              # new temperature proxy
    xi_t = corr_length_from_C(Psi)
    a_t  = float(xi_t/(xi0+1e-12))
    Phi  = (mean*mean)/(var+1e-12)
    Om   = Phi/(Theta_t+1e-12)

    t = step*DT
    ts.append(t); T_list.append(T_t); a_list.append(a_t)
    Theta_list.append(Theta_t); Phi_list.append(Phi); Om_list.append(Om)

    if SAVE_SNAPSHOTS and step in SNAPSHOT_STEPS:
        save_field(Psi, os.path.join(OUTDIR, f"psi_t{step}.png"), f"Ψ at t={t:.1f}")

# -------------- Save & Plot -----------------
df = pd.DataFrame({"t":ts,"T":T_list,"a":a_list,"Theta":Theta_list,"Phi":Phi_list,"Omega":Om_list})
df.to_csv(os.path.join(OUTDIR,"metrics_timeseries.csv"), index=False)

def simple_ma(x, w=21):
    w = max(3, int(w) | 1)
    pad = w//2
    xp = np.pad(x, (pad,pad), mode='edge')
    ker = np.ones(w)/w
    return np.convolve(xp, ker, mode='valid')

# Temperature
plt.figure(figsize=(7,4)); plt.plot(df["t"], simple_ma(df["T"], 31), linewidth=2)
plt.xlabel("time"); plt.ylabel("T ∝ ⟨|∇Ψ|²⟩"); plt.title("Cooling: T(t)")
plt.tight_layout(); plt.savefig(os.path.join(OUTDIR,"temperature_vs_time.png"),dpi=160); plt.close()

# Scale factor
plt.figure(figsize=(7,4)); plt.plot(df["t"], simple_ma(df["a"], 31), linewidth=2)
plt.xlabel("time"); plt.ylabel("a(t)=ξ/ξ₀"); plt.title("Expansion: a(t)")
plt.tight_layout(); plt.savefig(os.path.join(OUTDIR,"scale_factor_vs_time.png"),dpi=160); plt.close()

# Curvature
plt.figure(figsize=(7,4)); plt.plot(df["t"], simple_ma(df["Theta"], 31), linewidth=2)
plt.xlabel("time"); plt.ylabel("Θ(t)=⟨|∇²Ψ|⟩"); plt.title("Curvature Reduction")
plt.tight_layout(); plt.savefig(os.path.join(OUTDIR,"curvature_vs_time.png"),dpi=160); plt.close()

# Omega
plt.figure(figsize=(7,4)); plt.plot(df["t"], simple_ma(df["Omega"], 31), linewidth=2)
plt.xlabel("time"); plt.ylabel("Ω=Φ/Θ"); plt.title("CNT Ω(t)")
plt.tight_layout(); plt.savefig(os.path.join(OUTDIR,"omega_vs_time.png"),dpi=160); plt.close()

# Robust log–log slope of T vs a (middle 60% to avoid endpoints)
a = np.array(a_list); T = np.array(T_list)
mask = (a>0) & (T>0)
a, T = a[mask], T[mask]
p_lo, p_hi = np.percentile(np.arange(a.size), [20, 80]).astype(int)
idx = np.argsort(a)
loga = np.log(a[idx][p_lo:p_hi]); logT = np.log(T[idx][p_lo:p_hi])
slope, intercept = np.polyfit(loga, logT, 1)

plt.figure(figsize=(6,5))
plt.scatter(loga, logT, s=10)
xline = np.linspace(loga.min(), loga.max(), 200)
plt.plot(xline, slope*xline + intercept, linewidth=2)
plt.xlabel("log a"); plt.ylabel("log T"); plt.title(f"log T vs log a (robust slope ≈ {slope:.3f})")
plt.tight_layout(); plt.savefig(os.path.join(OUTDIR, "loglog_T_vs_a.png"), dpi=160); plt.close()

print("Saved:", OUTDIR)
for f in sorted(os.listdir(OUTDIR)): print(" -", f)


Saved: C:\Users\caleb\CNT_Lab\notebooks\archive\cnt_cosmo_drift_v2
 - curvature_vs_time.png
 - loglog_T_vs_a.png
 - metrics_timeseries.csv
 - omega_vs_time.png
 - psi_t0.png
 - psi_t1499.png
 - psi_t750.png
 - scale_factor_vs_time.png
 - temperature_vs_time.png


In [3]:
# CNT Cosmological Drift — BOTH MODES IN ONE GO (AC + CH)
# -------------------------------------------------------
# Runs Allen–Cahn (non-conserved) and Cahn–Hilliard (conserved) back-to-back
# with identical initialization. For each mode it saves:
#   - metrics_timeseries.csv
#   - T_grad(t), T_spec(t), a_k(t), k_peak(t), Theta(t), Omega(t)
#   - loglog_Tgrad_vs_ak.png, loglog_Tspec_vs_ak.png
#   - field snapshots at t=0, mid, end
# Also writes a combined "all_metrics.csv" with a Mode column, and prints a slope summary.
#
# Deps: numpy, pandas, matplotlib (no seaborn). One figure per plot. No explicit colors.

import os, math, numpy as np, pandas as pd, matplotlib.pyplot as plt
np.random.seed(7)

# ------------------------ Shared Controls ------------------------
ROOT = os.path.abspath("./cnt_cosmo_drift_both"); os.makedirs(ROOT, exist_ok=True)

# Grid & runtime
N, DX, DT, STEPS = 256, 1.0, 0.08, 2000
SNAPSHOT_STEPS = [0, STEPS//2, STEPS-1]
ROBUST_LO, ROBUST_HI = 0.2, 0.8  # middle fraction for robust slope

# Dynamics params (feel free to tweak)
# Allen–Cahn
AC_D       = 0.28
AC_LAMBDA  = 0.002
# Cahn–Hilliard
CH_KAPPA   = 0.9
CH_MU      = 0.0
CH_BETA    = 1.0

NOISE_AMP  = 0.0  # keep 0 for clarity

# ------------------------ Helpers ------------------------
def laplacian_periodic(X, dx):
    return (-4*X + np.roll(X,1,0)+np.roll(X,-1,0)+np.roll(X,1,1)+np.roll(X,-1,1))/(dx*dx)

def grad_energy(Psi, dx):
    gx = (np.roll(Psi,-1,1) - np.roll(Psi,1,1))/(2*dx)
    gy = (np.roll(Psi,-1,0) - np.roll(Psi,1,0))/(2*dx)
    return float(np.mean(gx*gx + gy*gy))

def radial_power(Psi):
    F = np.fft.fftn(Psi)
    S = (F*np.conj(F)).real
    S = np.fft.fftshift(S)
    ny,nx = S.shape; cy,cx = ny//2, nx//2
    y,x = np.indices((ny,nx))
    r = np.sqrt((y-cy)**2 + (x-cx)**2).astype(np.int32)
    rmax = min(cy,cx)
    sums = np.bincount(r.ravel(), weights=S.ravel(), minlength=rmax)
    cnts = np.bincount(r.ravel(), minlength=rmax)
    return np.where(cnts>0, sums/cnts, 0.0)

def k_from_bin(i, N, dx): 
    return 2.0*np.pi * i / (N*dx)

def k_peak(Psi, ignore_bins=1):
    Srad = radial_power(Psi)
    i0 = max(ignore_bins, 1)  # ignore DC
    i_max = i0 + int(np.argmax(Srad[i0:]))
    return k_from_bin(i_max, Psi.shape[0], DX)

def spectral_temperature(Psi):
    Srad = radial_power(Psi)
    ks = np.array([k_from_bin(i, Psi.shape[0], DX) for i in range(len(Srad))])
    num = np.sum((ks**2) * Srad)
    den = np.sum(Srad) + 1e-12
    return float(num/den)

def corr_length_from_C(Psi):
    F = np.fft.fftn(Psi); S = (F*np.conj(F)).real
    corr2d = np.fft.ifftn(S).real / (Psi.size)
    corr2d = np.fft.fftshift(corr2d)
    ny,nx = corr2d.shape; cy,cx = ny//2, nx//2
    y,x = np.indices((ny,nx)); r = np.sqrt((y-cy)**2 + (x-cx)**2).astype(np.int32)
    rmax = min(cy,cx)
    sums = np.bincount(r.ravel(), weights=corr2d.ravel(), minlength=rmax)
    cnts = np.bincount(r.ravel(), minlength=rmax)
    Cr = np.where(cnts>0, sums/cnts, 0.0)[:rmax]
    if Cr[0] <= 0: return 1.0
    Cr = Cr/Cr[0]; rr = np.arange(len(Cr))*DX
    i0 = max(2, len(rr)//200); i1 = max(i0+10, len(rr)//3)
    yv = np.clip(Cr[i0:i1], 1e-12, 1); xv = rr[i0:i1]
    slope, _ = np.polyfit(xv, np.log(yv), 1)
    return float(-1.0/slope if slope < 0 else 1.0)

def save_field(Psi, path, title):
    plt.figure(figsize=(6,6))
    im = plt.imshow(Psi, origin='lower', interpolation='nearest')
    plt.title(title); plt.colorbar(im, shrink=0.8)
    plt.tight_layout(); plt.savefig(path, dpi=160); plt.close()

def smooth_ma(x, w=41):
    w = max(5, int(w) | 1); pad = w//2
    xp = np.pad(x, (pad,pad), mode='edge'); ker = np.ones(w)/w
    return np.convolve(xp, ker, mode='valid')

def robust_loglog_slope(a, T, lo=0.2, hi=0.8):
    a = np.asarray(a); T = np.asarray(T)
    m = (a>0) & (T>0)
    a = a[m]; T = T[m]
    idx = np.argsort(a); a = a[idx]; T = T[idx]
    L = len(a)
    if L < 20: 
        return float('nan'), float('nan'), np.array([]), np.array([])
    i0, i1 = int(lo*L), max(int(hi*L), int(lo*L)+10)
    loga = np.log(a[i0:i1]); logT = np.log(T[i0:i1])
    s, b = np.polyfit(loga, logT, 1)
    return s, b, loga, logT

# ------------------------ Shared Initialization ------------------------
def make_initial_field():
    Psi = np.random.normal(0, 1.0, (N,N))
    F   = np.fft.fftn(Psi)
    ky  = np.fft.fftfreq(N, d=DX); kx = np.fft.fftfreq(N, d=DX)
    KX, KY = np.meshgrid(kx, ky, indexing='xy'); K2 = KX**2 + KY**2
    mask = np.exp(-K2/(0.08*(K2.max()+1e-12)))
    F *= (1.0 - 0.7*mask)    # suppress low-k modestly
    Psi = np.fft.ifftn(F).real
    return Psi*1.6

# ------------------------ Per-mode Runner ------------------------
def run_mode(mode_name, Psi0):
    outdir = os.path.join(ROOT, f"mode_{mode_name}")
    os.makedirs(outdir, exist_ok=True)
    Psi = Psi0.copy()

    # t=0 metrics
    var0  = float(np.var(Psi)); mean0 = float(np.mean(Psi))
    Tg0   = grad_energy(Psi, DX)
    Ts0   = spectral_temperature(Psi)
    xi0   = corr_length_from_C(Psi)
    k0    = k_peak(Psi)
    a0    = 1.0; ak0 = 1.0
    Theta0= float(np.mean(np.abs(laplacian_periodic(Psi, DX))))
    Phi0  = (mean0*mean0)/(var0+1e-12); Om0 = Phi0/(Theta0+1e-12)

    ts=[0.0]; Tgrad=[Tg0]; Tspec=[Ts0]; a_xi=[a0]; a_k=[ak0]
    Theta=[Theta0]; Phi=[Phi0]; Omega=[Om0]; ktrack=[k0]

    if 0 in SNAPSHOT_STEPS:
        save_field(Psi, os.path.join(outdir,"psi_t0.png"), f"Ψ ({mode_name}) at t=0")

    for step in range(STEPS):
        if mode_name == "AC":
            Lap = laplacian_periodic(Psi, DX)
            Theta_t = float(np.mean(np.abs(Lap)))
            Psi = Psi + DT*(AC_D*Lap - AC_LAMBDA*Psi**3)
        else:  # CH
            LapPsi = laplacian_periodic(Psi, DX)
            mu_c   = -CH_KAPPA*LapPsi + CH_BETA*(Psi**3) - CH_MU*Psi
            Theta_t= float(np.mean(np.abs(LapPsi)))
            Psi = Psi + DT*laplacian_periodic(mu_c, DX)

        if NOISE_AMP>0:
            Psi += np.random.normal(0, NOISE_AMP, size=Psi.shape)*math.sqrt(DT)

        var  = float(np.var(Psi)); mean = float(np.mean(Psi))
        Tg   = grad_energy(Psi, DX)
        Ts   = spectral_temperature(Psi)
        xi_t = corr_length_from_C(Psi)
        kp   = k_peak(Psi)
        ak   = float((k0+1e-12)/(kp+1e-12))
        ax   = float(xi_t/(xi0+1e-12))
        Phi_t= (mean*mean)/(var+1e-12)
        Om_t = Phi_t/(Theta_t+1e-12)

        t = (step+1)*DT
        ts.append(t); Tgrad.append(Tg); Tspec.append(Ts)
        a_xi.append(ax); a_k.append(ak); Theta.append(Theta_t)
        Phi.append(Phi_t); Omega.append(Om_t); ktrack.append(kp)

        if step in SNAPSHOT_STEPS:
            save_field(Psi, os.path.join(outdir, f"psi_t{step}.png"), f"Ψ ({mode_name}) at t={t:.1f}")

    # Save metrics
    df = pd.DataFrame({
        "t":ts, "T_grad":Tgrad, "T_spec":Tspec,
        "a_xi":a_xi, "a_k":a_k, "k_peak":ktrack,
        "Theta":Theta, "Phi":Phi, "Omega":Omega
    })
    df.to_csv(os.path.join(outdir,"metrics_timeseries.csv"), index=False)

    # Plots (one per figure)
    def pplot(x, y, xlabel, ylabel, title, fname):
        plt.figure(figsize=(7,4))
        plt.plot(x, y, linewidth=2)
        plt.xlabel(xlabel); plt.ylabel(ylabel); plt.title(title)
        plt.tight_layout(); plt.savefig(os.path.join(outdir,fname), dpi=160); plt.close()

    pplot(df["t"], smooth_ma(df["T_grad"], 41), "time", "T_grad ∝ ⟨|∇Ψ|²⟩",
          f"{mode_name}: Cooling T_grad(t)", "temperature_grad_vs_time.png")
    pplot(df["t"], smooth_ma(df["T_spec"], 41), "time", "T_spec ~ ⟨k²⟩_S",
          f"{mode_name}: Cooling T_spec(t)", "temperature_spec_vs_time.png")
    pplot(df["t"], df["a_k"], "time", "a_k = k₀/k_peak(t)",
          f"{mode_name}: Expansion a_k(t)", "scale_factor_ak_vs_time.png")
    pplot(df["t"], df["k_peak"], "time", "k_peak(t)",
          f"{mode_name}: Spectral Redshift k_peak(t)", "k_peak_vs_time.png")
    pplot(df["t"], smooth_ma(df["Theta"], 41), "time", "Θ = ⟨|∇²Ψ|⟩",
          f"{mode_name}: Curvature Θ(t)", "curvature_vs_time.png")
    pplot(df["t"], smooth_ma(df["Omega"], 41), "time", "Ω = Φ/Θ",
          f"{mode_name}: CNT Ω(t)", "omega_vs_time.png")

    # Robust slopes
    s_g, b_g, la_g, lt_g = robust_loglog_slope(df["a_k"], df["T_grad"], ROBUST_LO, ROBUST_HI)
    s_s, b_s, la_s, lt_s = robust_loglog_slope(df["a_k"], df["T_spec"], ROBUST_LO, ROBUST_HI)

    # Log–log plots
    def loglog_plot(loga, logT, s, b, ylabel, fname, title):
        plt.figure(figsize=(6,5))
        plt.scatter(loga, logT, s=10)
        xline = np.linspace(loga.min(), loga.max(), 200)
        plt.plot(xline, s*xline + b, linewidth=2)
        plt.xlabel("log a_k"); plt.ylabel(ylabel); plt.title(f"{title} (slope ≈ {s:.3f})")
        plt.tight_layout(); plt.savefig(os.path.join(outdir, fname), dpi=160); plt.close()

    if la_g.size:
        loglog_plot(la_g, lt_g, s_g, b_g, "log T_grad", "loglog_Tgrad_vs_ak.png",
                    f"{mode_name}: log T_grad vs log a_k")
    if la_s.size:
        loglog_plot(la_s, lt_s, s_s, b_s, "log T_spec", "loglog_Tspec_vs_ak.png",
                    f"{mode_name}: log T_spec vs log a_k")

    return df, {"Mode": mode_name, "slope_Tgrad_vs_ak": s_g, "slope_Tspec_vs_ak": s_s}

# ------------------------ Run BOTH ------------------------
Psi0 = make_initial_field()      # shared initial state
df_ac, res_ac = run_mode("AC", Psi0)
df_ch, res_ch = run_mode("CH", Psi0)

# Combine + save
df_ac_c = df_ac.copy(); df_ac_c["Mode"] = "AC"
df_ch_c = df_ch.copy(); df_ch_c["Mode"] = "CH"
df_all  = pd.concat([df_ac_c, df_ch_c], ignore_index=True)
df_all.to_csv(os.path.join(ROOT, "all_metrics.csv"), index=False)

# Print slope summary
summary = pd.DataFrame([res_ac, res_ch])
print("\n== CNT Cosmology Cooling Slopes (robust, middle band) ==")
print(summary.to_string(index=False))

print("\nArtifacts:")
print(" - Combined CSV:", os.path.join(ROOT, "all_metrics.csv"))
print(" - AC folder:", os.path.join(ROOT, "mode_AC"))
print(" - CH folder:", os.path.join(ROOT, "mode_CH"))


C:\Users\caleb\AppData\Local\Temp\ipykernel_47300\1068242250.py:110: RankWarning: Polyfit may be poorly conditioned
  s, b = np.polyfit(loga, logT, 1)
C:\Users\caleb\AppData\Local\Temp\ipykernel_47300\1068242250.py:110: RankWarning: Polyfit may be poorly conditioned
  s, b = np.polyfit(loga, logT, 1)
C:\Users\caleb\AppData\Local\Temp\ipykernel_47300\1068242250.py:153: RuntimeWarning: overflow encountered in power
  mu_c   = -CH_KAPPA*LapPsi + CH_BETA*(Psi**3) - CH_MU*Psi
C:\Users\caleb\AppData\Local\Temp\ipykernel_47300\1068242250.py:37: RuntimeWarning: overflow encountered in multiply
  return (-4*X + np.roll(X,1,0)+np.roll(X,-1,0)+np.roll(X,1,1)+np.roll(X,-1,1))/(dx*dx)
C:\Users\caleb\AppData\Local\Temp\ipykernel_47300\1068242250.py:37: RuntimeWarning: invalid value encountered in add
  return (-4*X + np.roll(X,1,0)+np.roll(X,-1,0)+np.roll(X,1,1)+np.roll(X,-1,1))/(dx*dx)
C:\Users\caleb\AppData\Local\Temp\ipykernel_47300\1068242250.py:37: RuntimeWarning: overflow encountered in add
  


== CNT Cosmology Cooling Slopes (robust, middle band) ==
Mode  slope_Tgrad_vs_ak  slope_Tspec_vs_ak
  AC          -1.277333          -0.510484
  CH                NaN                NaN

Artifacts:
 - Combined CSV: C:\Users\caleb\CNT_Lab\notebooks\archive\cnt_cosmo_drift_both\all_metrics.csv
 - AC folder: C:\Users\caleb\CNT_Lab\notebooks\archive\cnt_cosmo_drift_both\mode_AC
 - CH folder: C:\Users\caleb\CNT_Lab\notebooks\archive\cnt_cosmo_drift_both\mode_CH


In [4]:
# CNT Cosmological Drift — Phase Diagram Sweep (AC + CH)
# ------------------------------------------------------
# Sweeps parameters for both Allen–Cahn (AC) and Cahn–Hilliard (CH),
# computes robust slopes of log T_grad vs log a_k, and saves:
#  - ac_slopes.csv, ch_slopes.csv
#  - ac_slope_heatmap.png, ch_slope_heatmap.png
#  - a compact summary table printed at the end
#
# Runtime-friendly defaults: N=128, STEPS=900 (adjust as you like)

import os, math, numpy as np, pandas as pd, matplotlib.pyplot as plt
np.random.seed(7)

ROOT = os.path.abspath("./cnt_cosmo_phase_diagram"); os.makedirs(ROOT, exist_ok=True)

# ------------------- Global sim controls -------------------
N, DX, DT, STEPS = 128, 1.0, 0.08, 900
SNAPSHOT_STEPS = []          # empty ⇒ no field PNGs (keep sweep fast)
ROBUST_LO, ROBUST_HI = 0.25, 0.85
NOISE_AMP = 0.0

# AC sweep ranges (non-conserved)
AC_D_GRID      = np.linspace(0.18, 0.40, 5)     # diffusion
AC_LAMBDA_GRID = np.linspace(0.0012, 0.0040, 5) # nonlinearity

# CH sweep ranges (conserved)
CH_KAPPA_GRID  = np.linspace(0.6, 1.4, 5)       # interface stiffness
CH_BETA_GRID   = np.linspace(0.6, 1.4, 5)       # quartic strength
CH_MU = 0.0

# ------------------- Core helpers -------------------
def laplacian_periodic(X, dx):
    return (-4*X + np.roll(X,1,0)+np.roll(X,-1,0)+np.roll(X,1,1)+np.roll(X,-1,1))/(dx*dx)

def grad_energy(Psi, dx):
    gx = (np.roll(Psi,-1,1) - np.roll(Psi,1,1))/(2*dx)
    gy = (np.roll(Psi,-1,0) - np.roll(Psi,1,0))/(2*dx)
    return float(np.mean(gx*gx + gy*gy))

def radial_power(Psi):
    F = np.fft.fftn(Psi); S = (F*np.conj(F)).real
    S = np.fft.fftshift(S)
    ny,nx = S.shape; cy,cx = ny//2, nx//2
    y,x = np.indices((ny,nx))
    r = np.sqrt((y-cy)**2 + (x-cx)**2).astype(np.int32)
    rmax = min(cy,cx)
    sums = np.bincount(r.ravel(), weights=S.ravel(), minlength=rmax)
    cnts = np.bincount(r.ravel(), minlength=rmax)
    return np.where(cnts>0, sums/cnts, 0.0)

def k_from_bin(i, N, dx):
    return 2.0*np.pi * i / (N*dx)

def k_peak(Psi, ignore_bins=1):
    Srad = radial_power(Psi)
    i0 = max(ignore_bins, 1)
    i_max = i0 + int(np.argmax(Srad[i0:]))
    return k_from_bin(i_max, Psi.shape[0], DX)

def spectral_temperature(Psi):
    Srad = radial_power(Psi)
    ks = np.array([k_from_bin(i, Psi.shape[0], DX) for i in range(len(Srad))])
    num = np.sum((ks**2) * Srad); den = np.sum(Srad) + 1e-12
    return float(num/den)

def corr_length_from_C(Psi):
    F = np.fft.fftn(Psi); S = (F*np.conj(F)).real
    corr2d = np.fft.ifftn(S).real / (Psi.size)
    corr2d = np.fft.fftshift(corr2d)
    ny,nx = corr2d.shape; cy,cx = ny//2, nx//2
    y,x = np.indices((ny,nx)); r = np.sqrt((y-cy)**2 + (x-cx)**2).astype(np.int32)
    rmax = min(cy,cx)
    sums = np.bincount(r.ravel(), weights=corr2d.ravel(), minlength=rmax)
    cnts = np.bincount(r.ravel(), minlength=rmax)
    Cr = np.where(cnts>0, sums/cnts, 0.0)[:rmax]
    if Cr[0] <= 0: return 1.0
    Cr = Cr/Cr[0]; rr = np.arange(len(Cr))*DX
    i0 = max(2, len(rr)//200); i1 = max(i0+10, len(rr)//3)
    yv = np.clip(Cr[i0:i1], 1e-12, 1); xv = rr[i0:i1]
    slope, _ = np.polyfit(xv, np.log(yv), 1)
    return float(-1.0/slope if slope < 0 else 1.0)

def robust_loglog_slope(a, T, lo=0.25, hi=0.85):
    a = np.asarray(a); T = np.asarray(T)
    m = (a>0) & (T>0)
    a = a[m]; T = T[m]
    idx = np.argsort(a); a = a[idx]; T = T[idx]
    L = len(a)
    if L < 50: return float('nan')
    i0, i1 = int(lo*L), max(int(hi*L), int(lo*L)+10)
    loga = np.log(a[i0:i1]); logT = np.log(T[i0:i1])
    s, _ = np.polyfit(loga, logT, 1)
    return float(s)

def make_initial_field():
    Psi = np.random.normal(0, 1.0, (N,N))
    F   = np.fft.fftn(Psi)
    ky  = np.fft.fftfreq(N, d=DX); kx = np.fft.fftfreq(N, d=DX)
    KX, KY = np.meshgrid(kx, ky, indexing='xy'); K2 = KX**2 + KY**2
    mask = np.exp(-K2/(0.08*(K2.max()+1e-12)))
    F *= (1.0 - 0.7*mask)
    Psi = np.fft.ifftn(F).real
    return Psi*1.6

# ------------------- Single run (returns slope) -------------------
def run_and_slope(mode, params, Psi0):
    Psi = Psi0.copy()
    # t=0 anchors
    xi0 = corr_length_from_C(Psi)
    k0  = k_peak(Psi)
    a_k = [1.0]; Tgrad = [grad_energy(Psi, DX)]
    # evolve
    for step in range(STEPS):
        if mode=="AC":
            D, LAMBDA = params["D"], params["LAMBDA"]
            Lap = laplacian_periodic(Psi, DX)
            Psi = Psi + DT*(D*Lap - LAMBDA*Psi**3)
        else:
            KAPPA, BETA, MU = params["KAPPA"], params["BETA"], params["MU"]
            LapPsi = laplacian_periodic(Psi, DX)
            mu_c   = -KAPPA*LapPsi + BETA*(Psi**3) - MU*Psi
            Psi = Psi + DT*laplacian_periodic(mu_c, DX)
        if NOISE_AMP>0:
            Psi += np.random.normal(0, NOISE_AMP, size=Psi.shape)*math.sqrt(DT)
        kp = k_peak(Psi)
        a_k.append(float((k0+1e-12)/(kp+1e-12)))
        Tgrad.append(grad_energy(Psi, DX))
    return robust_loglog_slope(a_k, Tgrad, ROBUST_LO, ROBUST_HI)

# ------------------- Sweep AC -------------------
ac_records = []
for D in AC_D_GRID:
    for LAMBDA in AC_LAMBDA_GRID:
        Psi0 = make_initial_field()
        slope = run_and_slope("AC", {"D":float(D), "LAMBDA":float(LAMBDA)}, Psi0)
        ac_records.append({"Mode":"AC","D":float(D),"LAMBDA":float(LAMBDA),"slope_Tgrad_vs_ak":slope})
ac_df = pd.DataFrame(ac_records)
ac_csv = os.path.join(ROOT,"ac_slopes.csv"); ac_df.to_csv(ac_csv, index=False)

# Heatmap for AC (rows=LAMBDA, cols=D)
ac_pivot = ac_df.pivot(index="LAMBDA", columns="D", values="slope_Tgrad_vs_ak").sort_index(ascending=True)
plt.figure(figsize=(7,6))
plt.imshow(ac_pivot.values, aspect="auto")
plt.xticks(range(ac_pivot.shape[1]), [f"{c:.3f}" for c in ac_pivot.columns], rotation=45, ha="right")
plt.yticks(range(ac_pivot.shape[0]), [f"{r:.4f}" for r in ac_pivot.index])
plt.title("AC: slope(log T_grad) vs log a_k")
plt.xlabel("D"); plt.ylabel("LAMBDA")
plt.tight_layout(); plt.savefig(os.path.join(ROOT,"ac_slope_heatmap.png"), dpi=160); plt.close()

# ------------------- Sweep CH -------------------
ch_records = []
for KAPPA in CH_KAPPA_GRID:
    for BETA in CH_BETA_GRID:
        Psi0 = make_initial_field()
        slope = run_and_slope("CH", {"KAPPA":float(KAPPA),"BETA":float(BETA),"MU":float(CH_MU)}, Psi0)
        ch_records.append({"Mode":"CH","KAPPA":float(KAPPA),"BETA":float(BETA),"slope_Tgrad_vs_ak":slope})
ch_df = pd.DataFrame(ch_records)
ch_csv = os.path.join(ROOT,"ch_slopes.csv"); ch_df.to_csv(ch_csv, index=False)

# Heatmap for CH (rows=BETA, cols=KAPPA)
ch_pivot = ch_df.pivot(index="BETA", columns="KAPPA", values="slope_Tgrad_vs_ak").sort_index(ascending=True)
plt.figure(figsize=(7,6))
plt.imshow(ch_pivot.values, aspect="auto")
plt.xticks(range(ch_pivot.shape[1]), [f"{c:.2f}" for c in ch_pivot.columns], rotation=45, ha="right")
plt.yticks(range(ch_pivot.shape[0]), [f"{r:.2f}" for r in ch_pivot.index])
plt.title("CH: slope(log T_grad) vs log a_k")
plt.xlabel("KAPPA"); plt.ylabel("BETA")
plt.tight_layout(); plt.savefig(os.path.join(ROOT,"ch_slope_heatmap.png"), dpi=160); plt.close()

# ------------------- Print compact summary -------------------
def best_matches(df, target=-1.0, k=5):
    df = df.copy()
    df["abs_err"] = (df["slope_Tgrad_vs_ak"] - target).abs()
    return df.nsmallest(k, "abs_err")

print("\n== AC: slopes (near -1) ==")
print(best_matches(ac_df, -1.0, 6).drop(columns=["Mode"]).to_string(index=False))
print("\n== CH: slopes (near -1) ==")
print(best_matches(ch_df, -1.0, 6).drop(columns=["Mode"]).to_string(index=False))

print("\nArtifacts:")
print(" -", ac_csv)
print(" -", ch_csv)
print(" -", os.path.join(ROOT, "ac_slope_heatmap.png"))
print(" -", os.path.join(ROOT, "ch_slope_heatmap.png"))


C:\Users\caleb\AppData\Local\Temp\ipykernel_47300\1136630520.py:92: RankWarning: Polyfit may be poorly conditioned
  s, _ = np.polyfit(loga, logT, 1)
C:\Users\caleb\AppData\Local\Temp\ipykernel_47300\1136630520.py:92: RankWarning: Polyfit may be poorly conditioned
  s, _ = np.polyfit(loga, logT, 1)
C:\Users\caleb\AppData\Local\Temp\ipykernel_47300\1136630520.py:92: RankWarning: Polyfit may be poorly conditioned
  s, _ = np.polyfit(loga, logT, 1)
C:\Users\caleb\AppData\Local\Temp\ipykernel_47300\1136630520.py:92: RankWarning: Polyfit may be poorly conditioned
  s, _ = np.polyfit(loga, logT, 1)
C:\Users\caleb\AppData\Local\Temp\ipykernel_47300\1136630520.py:92: RankWarning: Polyfit may be poorly conditioned
  s, _ = np.polyfit(loga, logT, 1)
C:\Users\caleb\AppData\Local\Temp\ipykernel_47300\1136630520.py:92: RankWarning: Polyfit may be poorly conditioned
  s, _ = np.polyfit(loga, logT, 1)
C:\Users\caleb\AppData\Local\Temp\ipykernel_47300\1136630520.py:92: RankWarning: Polyfit may be poor


== AC: slopes (near -1) ==
    D  LAMBDA  slope_Tgrad_vs_ak  abs_err
0.345  0.0019          -1.000723 0.000723
0.180  0.0026          -0.987292 0.012708
0.180  0.0033          -0.986611 0.013389
0.235  0.0033          -0.976387 0.023613
0.235  0.0019          -1.046028 0.046028
0.290  0.0012          -1.108326 0.108326

== CH: slopes (near -1) ==
 KAPPA  BETA  slope_Tgrad_vs_ak  abs_err
   0.6   0.6                NaN      NaN
   0.6   0.8                NaN      NaN
   0.6   1.0                NaN      NaN
   0.6   1.2                NaN      NaN
   0.6   1.4                NaN      NaN
   0.8   0.6                NaN      NaN

Artifacts:
 - C:\Users\caleb\CNT_Lab\notebooks\archive\cnt_cosmo_phase_diagram\ac_slopes.csv
 - C:\Users\caleb\CNT_Lab\notebooks\archive\cnt_cosmo_phase_diagram\ch_slopes.csv
 - C:\Users\caleb\CNT_Lab\notebooks\archive\cnt_cosmo_phase_diagram\ac_slope_heatmap.png
 - C:\Users\caleb\CNT_Lab\notebooks\archive\cnt_cosmo_phase_diagram\ch_slope_heatmap.png


In [6]:
# CNT Cosmic Gates — Consolidated Report Cell (AC OK + CH centroid fix + summary)
# ------------------------------------------------------------------------------
# What it does:
# - Loads ac_slopes.csv / ch_slopes.csv if they exist (from your previous sweep)
# - Re-runs CH using a centroid-based scale factor a_c to avoid NaNs
# - Saves ch_slopes_centroid.csv and a heatmap
# - Prints best "near -1" gates for AC and CH
# - Writes CNT_cosmic_gates_report.md with a neat summary table

import os, math, numpy as np, pandas as pd, matplotlib.pyplot as plt
np.random.seed(7)

ROOT = os.path.abspath("./cnt_cosmo_phase_diagram")
os.makedirs(ROOT, exist_ok=True)

# ---------- Load prior AC / CH sweeps if present ----------
ac_csv = os.path.join(ROOT, "ac_slopes.csv")
ch_csv = os.path.join(ROOT, "ch_slopes.csv")
ac_df = pd.read_csv(ac_csv) if os.path.exists(ac_csv) else None
ch_df = pd.read_csv(ch_csv) if os.path.exists(ch_csv) else None

# ---------- CH centroid re-sweep params ----------
# (You can tweak grids; these match your previous ranges)
N, DX, DT, STEPS = 128, 1.0, 0.06, 1400
ROBUST_LO, ROBUST_HI = 0.25, 0.85
NOISE_AMP = 0.0
CH_KAPPA_GRID = np.linspace(0.6, 1.4, 5)
CH_BETA_GRID  = np.linspace(0.6, 1.4, 5)
CH_MU = 0.0

def laplacian_periodic(X, dx):
    return (-4*X + np.roll(X,1,0)+np.roll(X,-1,0)+np.roll(X,1,1)+np.roll(X,-1,1))/(dx*dx)

def radial_power(Psi):
    F = np.fft.fftn(Psi); S = (F*np.conj(F)).real
    S = np.fft.fftshift(S)
    ny,nx = S.shape; cy,cx = ny//2, nx//2
    y,x = np.indices((ny,nx))
    r = np.sqrt((y-cy)**2 + (x-cx)**2).astype(np.int32)
    rmax = min(cy,cx)
    sums = np.bincount(r.ravel(), weights=S.ravel(), minlength=rmax)
    cnts = np.bincount(r.ravel(), minlength=rmax)
    return np.where(cnts>0, sums/cnts, 0.0)

def k_from_bin(i, N, dx): 
    return 2.0*np.pi * i / (N*dx)

def spectral_moments(Psi):
    Srad = radial_power(Psi)
    ks = np.array([k_from_bin(i, Psi.shape[0], DX) for i in range(len(Srad))])
    Ssum = np.sum(Srad) + 1e-12
    k_mean = float(np.sum(ks*Srad) / Ssum)            # ⟨k⟩
    T_spec = float(np.sum((ks**2)*Srad) / Ssum)       # ∝ ⟨k²⟩
    return k_mean, T_spec

def grad_energy(Psi, dx):
    gx = (np.roll(Psi,-1,1) - np.roll(Psi,1,1))/(2*dx)
    gy = (np.roll(Psi,-1,0) - np.roll(Psi,1,0))/(2*dx)
    return float(np.mean(gx*gx + gy*gy))

def robust_loglog_slope(a, T, lo=0.25, hi=0.85):
    a = np.asarray(a); T = np.asarray(T)
    m = (a>0) & (T>0)
    a = a[m]; T = T[m]
    if a.size < 50: return float('nan')
    idx = np.argsort(a); a = a[idx]; T = T[idx]
    L = len(a); i0, i1 = int(lo*L), max(int(hi*L), int(lo*L)+10)
    loga = np.log(a[i0:i1]); logT = np.log(T[i0:i1])
    s, _ = np.polyfit(loga, logT, 1)
    return float(s)

def make_initial_field(n=N, dx=DX):
    Psi = np.random.normal(0, 1.0, (n,n))
    F   = np.fft.fftn(Psi)
    ky  = np.fft.fftfreq(n, d=dx); kx = np.fft.fftfreq(n, d=dx)
    KX, KY = np.meshgrid(kx, ky, indexing='xy'); K2 = KX**2 + KY**2
    mask = np.exp(-K2/(0.08*(K2.max()+1e-12)))
    F *= (1.0 - 0.7*mask)
    Psi = np.fft.ifftn(F).real
    return Psi*1.6

def run_ch_centroid_and_slope(KAPPA, BETA, MU, Psi0):
    Psi = Psi0.copy()
    k0, _ = spectral_moments(Psi)       # centroid at t=0
    a_c = [1.0]
    Tg  = [grad_energy(Psi, DX)]
    for _step in range(STEPS):
        LapPsi = laplacian_periodic(Psi, DX)
        mu_c   = -KAPPA*LapPsi + BETA*(Psi**3) - MU*Psi
        Psi    = Psi + DT*laplacian_periodic(mu_c, DX)
        if NOISE_AMP>0:
            Psi += np.random.normal(0, NOISE_AMP, size=Psi.shape)*math.sqrt(DT)
        k_mean, _Tspec = spectral_moments(Psi)
        a_c.append(float((k0+1e-12)/(k_mean+1e-12)))
        Tg.append(grad_energy(Psi, DX))
    # enforce weak monotonicity for safety
    a_c = np.maximum.accumulate(np.array(a_c))
    return robust_loglog_slope(a_c, Tg, ROBUST_LO, ROBUST_HI)

# ---------- Re-sweep CH using centroid a_c ----------
recs = []
for KAPPA in CH_KAPPA_GRID:
    for BETA in CH_BETA_GRID:
        Psi0 = make_initial_field()
        slope = run_ch_centroid_and_slope(float(KAPPA), float(BETA), float(CH_MU), Psi0)
        recs.append({"Mode":"CH_centroid","KAPPA":float(KAPPA),"BETA":float(BETA),"slope_Tgrad_vs_ac":slope})
ch_cent_df = pd.DataFrame(recs)

ch_cent_csv = os.path.join(ROOT, "ch_slopes_centroid.csv")
ch_cent_df.to_csv(ch_cent_csv, index=False)

# Heatmap for CH (centroid)
pivot = ch_cent_df.pivot(index="BETA", columns="KAPPA", values="slope_Tgrad_vs_ac").sort_index(ascending=True)
plt.figure(figsize=(7,6))
plt.imshow(pivot.values, aspect="auto")
plt.xticks(range(pivot.shape[1]), [f"{c:.2f}" for c in pivot.columns], rotation=45, ha="right")
plt.yticks(range(pivot.shape[0]), [f"{r:.2f}" for r in pivot.index])
plt.title("CH (centroid) — slope(log T_grad) vs log a_c")
plt.xlabel("KAPPA"); plt.ylabel("BETA")
plt.tight_layout()
heatmap_path = os.path.join(ROOT,"ch_slope_heatmap_centroid.png")
plt.savefig(heatmap_path, dpi=160); plt.close()

# ---------- Print best gates ----------
def best_matches(df, col, target=-1.0, k=6):
    out = df.copy()
    out["abs_err"] = (out[col] - target).abs()
    return out.nsmallest(k, "abs_err")

print("\n== AC: slopes (near -1) ==")
if ac_df is not None and "slope_Tgrad_vs_ak" in ac_df.columns:
    print(best_matches(ac_df, "slope_Tgrad_vs_ak", -1.0).drop(columns=["Mode","abs_err"]).to_string(index=False))
else:
    print("No ac_slopes.csv found — run the AC sweep cell first if needed.")

print("\n== CH (centroid): slopes (near -1) ==")
print(best_matches(ch_cent_df, "slope_Tgrad_vs_ac", -1.0).drop(columns=["Mode","abs_err"]).to_string(index=False))

print("\nArtifacts:")
print(" -", ac_csv if os.path.exists(ac_csv) else "(no AC file found)")
print(" -", ch_cent_csv)
print(" -", heatmap_path)

# ---------- Write Markdown report ----------
report = []
report.append("# CNT Cosmic Gates Report\n")
report.append("Radiation-like cooling corresponds to slope ≈ −1 in log T vs log a.\n")
if ac_df is not None and "slope_Tgrad_vs_ak" in ac_df.columns:
    top_ac = best_matches(ac_df, "slope_Tgrad_vs_ak", -1.0, k=3).reset_index(drop=True)
    report.append("## Allen–Cahn (non-conserved)\n")
    report.append(top_ac[["D","LAMBDA","slope_Tgrad_vs_ak"]].to_markdown(index=False))
else:
    report.append("## Allen–Cahn\nNo AC results found.\n")

top_ch = best_matches(ch_cent_df, "slope_Tgrad_vs_ac", -1.0, k=3).reset_index(drop=True)
report.append("\n## Cahn–Hilliard (conserved, centroid scale)\n")
report.append(top_ch[["KAPPA","BETA","slope_Tgrad_vs_ac"]].to_markdown(index=False))

report.append(f"\n### Artifacts\n- AC CSV: {ac_csv if os.path.exists(ac_csv) else '(not found)'}\n- CH centroid CSV: {ch_cent_csv}\n- CH centroid heatmap: {heatmap_path}\n")

with open(os.path.join(ROOT, "CNT_cosmic_gates_report.md"), "w", encoding="utf-8") as f:
    f.write("\n".join(report))

print("\nWrote report →", os.path.join(ROOT, "CNT_cosmo_phase_diagram", "CNT_cosmic_gates_report.md"))


C:\Users\caleb\AppData\Local\Temp\ipykernel_47300\1263547961.py:35: RuntimeWarning: overflow encountered in multiply
  F = np.fft.fftn(Psi); S = (F*np.conj(F)).real
C:\Users\caleb\AppData\Local\Temp\ipykernel_47300\1263547961.py:52: RuntimeWarning: invalid value encountered in multiply
  k_mean = float(np.sum(ks*Srad) / Ssum)            # ⟨k⟩
C:\Users\caleb\AppData\Local\Temp\ipykernel_47300\1263547961.py:53: RuntimeWarning: invalid value encountered in multiply
  T_spec = float(np.sum((ks**2)*Srad) / Ssum)       # ∝ ⟨k²⟩
C:\Users\caleb\AppData\Local\Temp\ipykernel_47300\1263547961.py:59: RuntimeWarning: overflow encountered in multiply
  return float(np.mean(gx*gx + gy*gy))
C:\Users\caleb\AppData\Local\Temp\ipykernel_47300\1263547961.py:59: RuntimeWarning: overflow encountered in add
  return float(np.mean(gx*gx + gy*gy))
C:\Users\caleb\AppData\Local\Temp\ipykernel_47300\1263547961.py:89: RuntimeWarning: overflow encountered in power
  mu_c   = -KAPPA*LapPsi + BETA*(Psi**3) - MU*Psi
C


== AC: slopes (near -1) ==
    D  LAMBDA  slope_Tgrad_vs_ak
0.345  0.0019          -1.000723
0.180  0.0026          -0.987292
0.180  0.0033          -0.986611
0.235  0.0033          -0.976387
0.235  0.0019          -1.046028
0.290  0.0012          -1.108326

== CH (centroid): slopes (near -1) ==
 KAPPA  BETA  slope_Tgrad_vs_ac
   0.6   0.6                NaN
   0.6   0.8                NaN
   0.6   1.0                NaN
   0.6   1.2                NaN
   0.6   1.4                NaN
   0.8   0.6                NaN

Artifacts:
 - C:\Users\caleb\CNT_Lab\notebooks\archive\cnt_cosmo_phase_diagram\ac_slopes.csv
 - C:\Users\caleb\CNT_Lab\notebooks\archive\cnt_cosmo_phase_diagram\ch_slopes_centroid.csv
 - C:\Users\caleb\CNT_Lab\notebooks\archive\cnt_cosmo_phase_diagram\ch_slope_heatmap_centroid.png


ImportError: Missing optional dependency 'tabulate'.  Use pip or conda to install tabulate.

In [7]:
# CNT — CH Slope Fix (Centroid a_c) + No-Tabulate Report
# ------------------------------------------------------
# * Forces robust CH coarsening with MU=1.0 (ψ^3 − ψ double-well)
# * Longer run + tiny noise -> centroid shifts cleanly
# * Writes ch_slopes_centroid.csv + heatmap (no seaborn)
# * Reads existing ac_slopes.csv if present
# * Prints best gates and writes a markdown report WITHOUT needing 'tabulate'

import os, math, numpy as np, pandas as pd, matplotlib.pyplot as plt
np.random.seed(7)

ROOT = os.path.abspath("./cnt_cosmo_phase_diagram"); os.makedirs(ROOT, exist_ok=True)

# ==== CH sweep params (beefed up for coarsening) ====
N, DX = 128, 1.0
DT, STEPS = 0.05, 1600     # longer + slightly smaller dt than before
ROBUST_LO, ROBUST_HI = 0.25, 0.85
NOISE_AMP = 5e-4           # tiny symmetry-breaking noise
CH_KAPPA_GRID = np.linspace(0.6, 1.4, 5)
CH_BETA_GRID  = np.linspace(0.8, 1.6, 5)
CH_MU = 1.0                # KEY: ψ^3 − ψ  (spinodal-friendly)

# ==== Helpers ====
def laplacian_periodic(X, dx):
    return (-4*X + np.roll(X,1,0)+np.roll(X,-1,0)+np.roll(X,1,1)+np.roll(X,-1,1))/(dx*dx)

def radial_power(Psi):
    F = np.fft.fftn(Psi); S = (F*np.conj(F)).real
    S = np.fft.fftshift(S)
    ny,nx = S.shape; cy,cx = ny//2, nx//2
    y,x = np.indices((ny,nx))
    r = np.sqrt((y-cy)**2 + (x-cx)**2).astype(np.int32)
    rmax = min(cy,cx)
    sums = np.bincount(r.ravel(), weights=S.ravel(), minlength=rmax)
    cnts = np.bincount(r.ravel(), minlength=rmax)
    return np.where(cnts>0, sums/cnts, 0.0)

def k_from_bin(i, N, dx): 
    return 2.0*np.pi * i / (N*dx)

def spectral_moments(Psi):
    Srad = radial_power(Psi)
    ks = np.array([k_from_bin(i, Psi.shape[0], DX) for i in range(len(Srad))])
    Ssum = np.sum(Srad) + 1e-12
    k_mean = float(np.sum(ks*Srad) / Ssum)            # ⟨k⟩
    T_spec = float(np.sum((ks**2)*Srad) / Ssum)       # ∝ ⟨k²⟩
    return k_mean, T_spec

def grad_energy(Psi, dx):
    gx = (np.roll(Psi,-1,1) - np.roll(Psi,1,1))/(2*dx)
    gy = (np.roll(Psi,-1,0) - np.roll(Psi,1,0))/(2*dx)
    return float(np.mean(gx*gx + gy*gy))

def robust_loglog_slope(a, T, lo=0.25, hi=0.85):
    a = np.asarray(a); T = np.asarray(T)
    m = (a>0) & (T>0)
    a = a[m]; T = T[m]
    if a.size < 60:  # need enough samples to fit reliably
        return float('nan')
    idx = np.argsort(a); a = a[idx]; T = T[idx]
    L = len(a); i0, i1 = int(lo*L), max(int(hi*L), int(lo*L)+12)
    if i1 - i0 < 12:
        return float('nan')
    loga = np.log(a[i0:i1]); logT = np.log(T[i0:i1])
    s, _ = np.polyfit(loga, logT, 1)
    return float(s)

def make_initial_field(n=N, dx=DX):
    # Slightly stronger high-k emphasis and amplitude to seed spinodal growth
    Psi = np.random.normal(0, 1.0, (n,n))
    F   = np.fft.fftn(Psi)
    ky  = np.fft.fftfreq(n, d=dx); kx = np.fft.fftfreq(n, d=dx)
    KX, KY = np.meshgrid(kx, ky, indexing='xy'); K2 = KX**2 + KY**2
    mask = np.exp(-K2/(0.06*(K2.max()+1e-12)))   # tighter low-k suppression
    F *= (1.0 - 0.8*mask)
    Psi = np.fft.ifftn(F).real
    return Psi*1.9

def run_ch_centroid_and_slope(KAPPA, BETA, MU, Psi0):
    Psi = Psi0.copy()
    k0, _ = spectral_moments(Psi)     # centroid at t=0
    a_c = [1.0]
    Tg  = [grad_energy(Psi, DX)]
    for _ in range(STEPS):
        LapPsi = laplacian_periodic(Psi, DX)
        mu_c   = -KAPPA*LapPsi + BETA*(Psi**3) - MU*Psi   # ψ^3 − ψ with stiffness
        Psi    = Psi + DT*laplacian_periodic(mu_c, DX)
        if NOISE_AMP>0:
            Psi += np.random.normal(0, NOISE_AMP, size=Psi.shape)*math.sqrt(DT)
        k_mean, _Tspec = spectral_moments(Psi)
        a_c.append(float((k0+1e-12)/(k_mean+1e-12)))
        Tg.append(grad_energy(Psi, DX))
    # enforce weak monotonicity (rare small jiggles)
    a_c = np.maximum.accumulate(np.array(a_c))
    return robust_loglog_slope(a_c, Tg, ROBUST_LO, ROBUST_HI)

# ==== Run CH sweep with centroid scale ====
recs = []
for KAPPA in CH_KAPPA_GRID:
    for BETA in CH_BETA_GRID:
        Psi0 = make_initial_field()
        slope = run_ch_centroid_and_slope(float(KAPPA), float(BETA), float(CH_MU), Psi0)
        recs.append({"Mode":"CH_centroid","KAPPA":float(KAPPA),"BETA":float(BETA),
                     "MU":float(CH_MU),"slope_Tgrad_vs_ac":slope})
ch_cent_df = pd.DataFrame(recs)
ch_cent_csv = os.path.join(ROOT, "ch_slopes_centroid.csv")
ch_cent_df.to_csv(ch_cent_csv, index=False)

# ==== Heatmap (no seaborn) ====
pivot = ch_cent_df.pivot(index="BETA", columns="KAPPA", values="slope_Tgrad_vs_ac").sort_index(ascending=True)
plt.figure(figsize=(7,6))
plt.imshow(pivot.values, aspect="auto")
plt.xticks(range(pivot.shape[1]), [f"{c:.2f}" for c in pivot.columns], rotation=45, ha="right")
plt.yticks(range(pivot.shape[0]), [f"{r:.2f}" for r in pivot.index])
plt.title("CH (centroid) — slope(log T_grad) vs log a_c")
plt.xlabel("KAPPA"); plt.ylabel("BETA")
plt.tight_layout()
heatmap_path = os.path.join(ROOT,"ch_slope_heatmap_centroid.png")
plt.savefig(heatmap_path, dpi=160); plt.close()

# ==== Load AC if present ====
ac_csv = os.path.join(ROOT, "ac_slopes.csv")
ac_df = pd.read_csv(ac_csv) if os.path.exists(ac_csv) else None

# ==== Print best gates (AC near −1, CH near −1) ====
def best_matches(df, col, target=-1.0, k=6):
    out = df.copy()
    out["abs_err"] = (out[col] - target).abs()
    return out.nsmallest(k, "abs_err")

print("\n== AC: slopes (near -1) ==")
if ac_df is not None and "slope_Tgrad_vs_ak" in ac_df.columns:
    print(best_matches(ac_df, "slope_Tgrad_vs_ak", -1.0)[["D","LAMBDA","slope_Tgrad_vs_ak"]].to_string(index=False))
else:
    print("(no ac_slopes.csv found here)")

print("\n== CH (centroid): slopes (near -1) ==")
print(best_matches(ch_cent_df, "slope_Tgrad_vs_ac", -1.0)[["KAPPA","BETA","MU","slope_Tgrad_vs_ac"]].to_string(index=False))

# ==== Write simple Markdown report WITHOUT tabulate ====
def df_to_md(df, cols):
    lines = []
    header = "| " + " | ".join(cols) + " |"
    sep    = "| " + " | ".join(["---"]*len(cols)) + " |"
    lines.append(header); lines.append(sep)
    for _, row in df.iterrows():
        lines.append("| " + " | ".join(str(row[c]) for c in cols) + " |")
    return "\n".join(lines)

report_lines = ["# CNT Cosmic Gates Report",
                "Target: radiation-like cooling (slope ≈ −1) in log T vs log a.\n"]

if ac_df is not None and "slope_Tgrad_vs_ak" in ac_df.columns:
    top_ac = best_matches(ac_df, "slope_Tgrad_vs_ak", -1.0, k=3)[["D","LAMBDA","slope_Tgrad_vs_ak"]]
    report_lines += ["## Allen–Cahn (non-conserved)", df_to_md(top_ac, ["D","LAMBDA","slope_Tgrad_vs_ak"]), ""]
else:
    report_lines += ["## Allen–Cahn (non-conserved)", "_No AC sweep CSV found in this folder._", ""]

top_ch = best_matches(ch_cent_df, "slope_Tgrad_vs_ac", -1.0, k=3)[["KAPPA","BETA","MU","slope_Tgrad_vs_ac"]]
report_lines += ["## Cahn–Hilliard (conserved, centroid scale)",
                 df_to_md(top_ch, ["KAPPA","BETA","MU","slope_Tgrad_vs_ac"]), ""]

report_lines += ["### Artifacts",
                 f"- AC CSV: {ac_csv if os.path.exists(ac_csv) else '(not found)'}",
                 f"- CH centroid CSV: {ch_cent_csv}",
                 f"- CH centroid heatmap: {heatmap_path}"]

md_path = os.path.join(ROOT, "CNT_cosmic_gates_report.md")
with open(md_path, "w", encoding="utf-8") as f:
    f.write("\n".join(report_lines))

print("\nArtifacts:")
print(" -", ch_cent_csv)
print(" -", heatmap_path)
print(" -", md_path)


C:\Users\caleb\AppData\Local\Temp\ipykernel_47300\577087078.py:86: RuntimeWarning: overflow encountered in power
  mu_c   = -KAPPA*LapPsi + BETA*(Psi**3) - MU*Psi   # ψ^3 − ψ with stiffness
C:\Users\caleb\AppData\Local\Temp\ipykernel_47300\577087078.py:25: RuntimeWarning: overflow encountered in multiply
  return (-4*X + np.roll(X,1,0)+np.roll(X,-1,0)+np.roll(X,1,1)+np.roll(X,-1,1))/(dx*dx)
C:\Users\caleb\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\fft\_pocketfft.py:101: RuntimeWarning: invalid value encountered in fft
  return ufunc(a, fct, axes=[(axis,), (), (axis,)], out=out)
C:\Users\caleb\AppData\Local\Temp\ipykernel_47300\577087078.py:28: RuntimeWarning: invalid value encountered in multiply
  F = np.fft.fftn(Psi); S = (F*np.conj(F)).real
C:\Users\caleb\AppData\Local\Temp\ipykernel_47300\577087078.py:50: RuntimeWarning: invalid value encountered in subtract
  gx = (np.roll(Psi,-1,1) - np.roll(Psi,1,1))/(2*dx)
C:\Users\caleb\AppData\Local\Temp\ipykernel_47300\5


== AC: slopes (near -1) ==
    D  LAMBDA  slope_Tgrad_vs_ak
0.345  0.0019          -1.000723
0.180  0.0026          -0.987292
0.180  0.0033          -0.986611
0.235  0.0033          -0.976387
0.235  0.0019          -1.046028
0.290  0.0012          -1.108326

== CH (centroid): slopes (near -1) ==
 KAPPA  BETA  MU  slope_Tgrad_vs_ac
   0.6   0.8 1.0                NaN
   0.6   1.0 1.0                NaN
   0.6   1.2 1.0                NaN
   0.6   1.4 1.0                NaN
   0.6   1.6 1.0                NaN
   0.8   0.8 1.0                NaN

Artifacts:
 - C:\Users\caleb\CNT_Lab\notebooks\archive\cnt_cosmo_phase_diagram\ch_slopes_centroid.csv
 - C:\Users\caleb\CNT_Lab\notebooks\archive\cnt_cosmo_phase_diagram\ch_slope_heatmap_centroid.png
 - C:\Users\caleb\CNT_Lab\notebooks\archive\cnt_cosmo_phase_diagram\CNT_cosmic_gates_report.md


In [8]:
# Stable Cahn–Hilliard (CH) — Spectral Semi-Implicit (Convex-Splitting)
# Produces: ch_centroid_metrics.csv, slope, and plots without overflows.

import os, math, numpy as np, pandas as pd, matplotlib.pyplot as plt
np.seterr(all='ignore')  # we'll keep things numerically safe by construction
np.random.seed(7)

OUTDIR = os.path.abspath("./cnt_ch_stable"); os.makedirs(OUTDIR, exist_ok=True)

# ---------------- Params (tweak if desired) ----------------
N, L = 256, 256.0       # grid points and box length
DX = L / N
DT = 0.08               # stable with semi-implicit; you can raise to 0.12 if desired
STEPS = 1800
SAVE_EVERY = 300
KAPPA = 1.0             # interface stiffness (κ)
BETA  = 1.0             # quartic weight on ψ^3 term
MU    = 1.0             # double-well bias: f'(ψ)=β ψ^3 - μ ψ
NOISE_AMP = 1e-3        # tiny noise to break symmetry

# ---------------- Grids & spectral operators ----------------
x = np.arange(N)*DX; y = np.arange(N)*DX
kx = 2.0*np.pi*np.fft.fftfreq(N, d=DX)
ky = 2.0*np.pi*np.fft.fftfreq(N, d=DX)
KX, KY = np.meshgrid(kx, ky, indexing='xy')
K2 = KX**2 + KY**2
Lop = K2  # since ∇² -> -K2 in Fourier, we’ll use K2 with signs carefully
den = 1.0 + DT * (K2) * (KAPPA * K2 + MU)   # denominator for semi-implicit update (always >= 1)

# 2/3 dealias mask
kx_cut = (2.0/3.0) * (np.max(np.abs(kx)))
ky_cut = (2.0/3.0) * (np.max(np.abs(ky)))
dealias = (np.abs(KX) <= kx_cut) & (np.abs(KY) <= ky_cut)

# ---------------- Helpers ----------------
def grad_energy(Psi):
    gx = (np.roll(Psi,-1,1) - np.roll(Psi,1,1))/(2*DX)
    gy = (np.roll(Psi,-1,0) - np.roll(Psi,1,0))/(2*DX)
    return float(np.mean(gx*gx + gy*gy))

def spectral_power(Psi):
    F = np.fft.fftn(Psi)
    S = (F*np.conj(F)).real
    S = np.fft.fftshift(S)
    return S

def centroid_scale(Psi, k_mean0=None):
    S = spectral_power(Psi)
    # radial average of S(k): compute <k> = sum k*S / sum S
    Sshift = S
    # build |k| grid (shifted)
    KXsh = np.fft.fftshift(KX); KYsh = np.fft.fftshift(KY)
    Kmag = np.sqrt(KXsh**2 + KYsh**2)
    Ssum = np.sum(Sshift) + 1e-12
    k_mean = float(np.sum(Kmag * Sshift) / Ssum)
    if k_mean0 is None:
        return 1.0, k_mean
    a_c = float((k_mean0 + 1e-12) / (k_mean + 1e-12))
    return a_c, k_mean

def save_field(Psi, path, title):
    plt.figure(figsize=(6,6))
    im = plt.imshow(Psi, origin='lower', interpolation='nearest')
    plt.title(title); plt.colorbar(im, shrink=0.8)
    plt.tight_layout(); plt.savefig(path, dpi=160); plt.close()

# ---------------- Initialization ----------------
Psi = np.random.normal(0, 1.0, (N,N))
# emphasize mid/high-k a bit, but keep amplitudes moderate to avoid initial spikes
F   = np.fft.fftn(Psi)
K2max = np.max(K2) + 1e-12
mask = np.exp(-K2/(0.10*K2max))
F *= (1.0 - 0.6*mask)
F *= dealias
Psi = np.fft.ifftn(F).real
Psi *= 1.2
Psi += np.random.normal(0, NOISE_AMP, size=Psi.shape)

# ---------------- Anchors & logs ----------------
a_list = []
Tgrad_list = []
t_list = []

a0, k0 = centroid_scale(Psi, None)
a_list.append(1.0); Tgrad_list.append(grad_energy(Psi)); t_list.append(0.0)

save_field(Psi, os.path.join(OUTDIR, f"psi_t0000.png"), "Ψ (CH) t=0")

# ---------------- Semi-implicit CH loop ----------------
for step in range(1, STEPS+1):
    # Explicit nonlinearity in real space
    Psi3 = Psi*Psi*Psi

    # FFTs (dealiased)
    F_Psi3 = np.fft.fftn(Psi3); F_Psi3 *= dealias

    # Right-hand side in Fourier: ψ^n + Δt * ∇²( β ψ^3 )
    # ∇² in Fourier is -K2, so ∇²(β ψ^3) -> -K2 * β * FFT(ψ^3)
    RHS = np.fft.fftn(Psi) + DT * (-K2) * (BETA * F_Psi3)

    # Update (implicit on linear part): ψ^{n+1} = RHS / [1 + Δt * K2 * (κ K2 + μ)]
    F_next = RHS / den
    F_next *= dealias
    Psi = np.fft.ifftn(F_next).real

    # tiny boundedness guard (prevents rare runaway due to rounding)
    Psi = np.clip(Psi, -5.0, 5.0)

    if NOISE_AMP > 0 and (step % 50 == 0):
        Psi += np.random.normal(0, NOISE_AMP, size=Psi.shape)

    # Metrics
    a_c, k_mean = centroid_scale(Psi, k0)
    a_list.append(a_c)
    Tgrad_list.append(grad_energy(Psi))
    t_list.append(step*DT)

    if step % SAVE_EVERY == 0 or step == STEPS:
        save_field(Psi, os.path.join(OUTDIR, f"psi_t{step:04d}.png"), f"Ψ (CH) t={step*DT:.1f}")

# enforce weak monotonicity on a_c (just in case of tiny numerical jiggles)
a_arr = np.maximum.accumulate(np.array(a_list))
T_arr = np.array(Tgrad_list)
t_arr = np.array(t_list)

# ---------------- Slope fit (log T vs log a_c) ----------------
mask = (a_arr > 0) & (T_arr > 0)
a_use = a_arr[mask]; T_use = T_arr[mask]
order = np.argsort(a_use); a_use = a_use[order]; T_use = T_use[order]
L = len(a_use); lo, hi = int(0.25*L), max(int(0.85*L), int(0.25*L)+12)
loga = np.log(a_use[lo:hi]); logT = np.log(T_use[lo:hi])
slope, intercept = np.polyfit(loga, logT, 1) if loga.size >= 12 else (np.nan, np.nan)

# ---------------- Save metrics & plots ----------------
df = pd.DataFrame({"t": t_arr, "a_c": a_arr, "T_grad": T_arr})
df.to_csv(os.path.join(OUTDIR, "ch_centroid_metrics.csv"), index=False)

plt.figure(figsize=(7,4))
plt.plot(t_arr, a_arr, linewidth=2)
plt.xlabel("time"); plt.ylabel("a_c = ⟨k⟩(0)/⟨k⟩(t)")
plt.title("CH Expansion via Spectral Centroid a_c(t)")
plt.tight_layout(); plt.savefig(os.path.join(OUTDIR, "a_c_vs_time.png"), dpi=160); plt.close()

plt.figure(figsize=(7,4))
plt.plot(t_arr, T_arr, linewidth=2)
plt.xlabel("time"); plt.ylabel("T_grad ∝ ⟨|∇Ψ|²⟩")
plt.title("CH Cooling: T_grad(t)")
plt.tight_layout(); plt.savefig(os.path.join(OUTDIR, "Tgrad_vs_time.png"), dpi=160); plt.close()

plt.figure(figsize=(6,5))
plt.scatter(loga, logT, s=10)
xline = np.linspace(loga.min(), loga.max(), 200)
plt.plot(xline, slope*xline + intercept, linewidth=2)
plt.xlabel("log a_c"); plt.ylabel("log T_grad")
plt.title(f"CH: log T vs log a_c (slope ≈ {slope:.3f})")
plt.tight_layout(); plt.savefig(os.path.join(OUTDIR, "loglog_T_vs_ac.png"), dpi=160); plt.close()

print("Saved to:", OUTDIR)
print("Slope (log T_grad vs log a_c) ≈", slope)


Saved to: C:\Users\caleb\CNT_Lab\notebooks\archive\cnt_ch_stable
Slope (log T_grad vs log a_c) ≈ -3.923292874207222


In [9]:
# CH Slope Triage — try multiple rulers (a_corr, a_peak, a_len) and thermometers (T_grad, T_spec)
import os, math, numpy as np, pandas as pd, matplotlib.pyplot as plt
np.seterr(all='ignore'); np.random.seed(7)

OUTDIR = os.path.abspath("./cnt_ch_slope_triage"); os.makedirs(OUTDIR, exist_ok=True)

# ===== Semi-implicit CH integrator (stable) =====
N, L = 256, 256.0
DX = L / N
DT = 0.08
STEPS = 1400
SAVE_EVERY = 999999  # no PNGs this time (fast)
KAPPA, BETA, MU = 1.0, 1.0, 1.0
NOISE_AMP = 1e-3

kx = 2.0*np.pi*np.fft.fftfreq(N, d=DX)
ky = 2.0*np.pi*np.fft.fftfreq(N, d=DX)
KX, KY = np.meshgrid(kx, ky, indexing='xy')
K2 = KX**2 + KY**2
den = 1.0 + DT * (K2) * (KAPPA * K2 + MU)

kx_cut = (2/3)*np.max(np.abs(kx))
ky_cut = (2/3)*np.max(np.abs(ky))
dealias = (np.abs(KX) <= kx_cut) & (np.abs(KY) <= ky_cut)

def grad_energy(Psi):
    gx = (np.roll(Psi,-1,1) - np.roll(Psi,1,1))/(2*DX)
    gy = (np.roll(Psi,-1,0) - np.roll(Psi,1,0))/(2*DX)
    return float(np.mean(gx*gx + gy*gy))

def spectral_power(Psi):
    F = np.fft.fftn(Psi)
    S = (F*np.conj(F)).real
    return np.fft.fftshift(S)

def radialize(S):
    ny,nx = S.shape; cy,cx = ny//2, nx//2
    y,x = np.indices((ny,nx))
    r = np.sqrt((y-cy)**2 + (x-cx)**2).astype(np.int32)
    rmax = min(cy,cx)
    sums = np.bincount(r.ravel(), weights=S.ravel(), minlength=rmax)
    cnts = np.bincount(r.ravel(), minlength=rmax)
    return np.where(cnts>0, sums/cnts, 0.0)

def k_from_bin(i): return 2.0*np.pi * i / (N*DX)

def k_peak(Psi):
    Srad = radialize(spectral_power(Psi))
    i0 = 1
    i_max = i0 + int(np.argmax(Srad[i0:]))
    return k_from_bin(i_max)

def spectral_moments(Psi):
    Srad = radialize(spectral_power(Psi))
    ks = np.array([k_from_bin(i) for i in range(len(Srad))])
    Ssum = np.sum(Srad) + 1e-12
    k_mean = float(np.sum(ks*Srad)/Ssum)
    k2_mean= float(np.sum((ks**2)*Srad)/Ssum)  # this is T_spec
    return k_mean, k2_mean

def corr_length(Psi):
    # ξ from autocorrelation C(r) ~ exp(-r/ξ)
    F = np.fft.fftn(Psi); S = (F*np.conj(F)).real
    C = np.fft.ifftn(S).real / Psi.size
    C = np.fft.fftshift(C)
    # radial avg of C
    Crad = radialize(C)
    if Crad[0] <= 0: return 1.0
    Cn = Crad / Crad[0]
    rr = np.arange(len(Cn))*DX
    i0 = max(2, len(rr)//200); i1 = max(i0+10, len(rr)//3)
    y = np.clip(Cn[i0:i1], 1e-12, 1); x = rr[i0:i1]
    s,_ = np.polyfit(x, np.log(y), 1)
    return float(-1.0/s) if s < 0 else 1.0

def interface_length(Psi, tau=0.25):
    # crude: count pixels where |∇ψ| exceeds threshold; proportional to total interface length
    gx = (np.roll(Psi,-1,1) - np.roll(Psi,1,1))/(2*DX)
    gy = (np.roll(Psi,-1,0) - np.roll(Psi,1,0))/(2*DX)
    G = np.sqrt(gx*gx + gy*gy)
    return float(np.mean(G > tau))

def robust_slope(a, T, lo=0.25, hi=0.85):
    a = np.asarray(a); T = np.asarray(T)
    m = (a>0) & (T>0)
    a = a[m]; T = T[m]
    if a.size < 60: return np.nan
    idx = np.argsort(a); a = a[idx]; T = T[idx]
    L = len(a); i0, i1 = int(lo*L), max(int(hi*L), int(lo*L)+12)
    loga = np.log(a[i0:i1]); logT = np.log(T[i0:i1])
    s,_ = np.polyfit(loga, logT, 1) if loga.size>=12 else (np.nan, np.nan)
    return float(s)

# ===== Initialize =====
Psi = np.random.normal(0, 1.0, (N,N))
F = np.fft.fftn(Psi)
K2max = np.max(K2) + 1e-12
mask = np.exp(-K2/(0.10*K2max))
F *= (1.0 - 0.6*mask)
F *= dealias
Psi = np.fft.ifftn(F).real
Psi *= 1.2
Psi += np.random.normal(0, NOISE_AMP, size=Psi.shape)

# ===== Logs =====
t = 0.0
Tgrad, Tspec = [], []
a_corr, a_peak, a_len = [], [], []

xi0 = corr_length(Psi)
kp0 = k_peak(Psi)
L0  = interface_length(Psi)

def push_logs(Psi, t):
    Tgrad.append(grad_energy(Psi))
    _, k2 = spectral_moments(Psi); Tspec.append(k2)
    xi = corr_length(Psi); kp = k_peak(Psi); L = interface_length(Psi)
    a_corr.append(xi / (xi0 + 1e-12))
    a_peak.append((kp0 + 1e-12) / (kp + 1e-12))
    a_len.append((L0 + 1e-12) / (L + 1e-12))

push_logs(Psi, t)

# ===== Evolve (semi-implicit) =====
for step in range(1, STEPS+1):
    Psi3 = Psi*Psi*Psi
    F_Psi3 = np.fft.fftn(Psi3); F_Psi3 *= dealias
    RHS = np.fft.fftn(Psi) + DT * (-K2) * (BETA * F_Psi3)
    F_next = RHS / den
    F_next *= dealias
    Psi = np.fft.ifftn(F_next).real
    Psi = np.clip(Psi, -5.0, 5.0)
    if NOISE_AMP>0 and step % 50 == 0:
        Psi += np.random.normal(0, NOISE_AMP, size=Psi.shape)
    if step % 5 == 0:  # log every few steps
        t += 5*DT
        push_logs(Psi, t)

# ===== Slopes =====
s_Tg_acorr  = robust_slope(a_corr, Tgrad)
s_Ts_acorr  = robust_slope(a_corr, Tspec)
s_Tg_apeak  = robust_slope(a_peak, Tgrad)
s_Tg_alen   = robust_slope(a_len,  Tgrad)
s_Ts_apeak  = robust_slope(a_peak, Tspec)

print("\n== CH slope triage (target ~ -1) ==")
print(f"T_grad vs a_corr : {s_Tg_acorr:.3f}")
print(f"T_spec vs a_corr : {s_Ts_acorr:.3f}")
print(f"T_grad vs a_peak : {s_Tg_apeak:.3f}")
print(f"T_grad vs a_len  : {s_Tg_alen:.3f}")
print(f"T_spec vs a_peak : {s_Ts_apeak:.3f}")

# Save CSV
df = pd.DataFrame({
    "t": np.linspace(0, t, len(a_corr)),
    "a_corr": a_corr, "a_peak": a_peak, "a_len": a_len,
    "T_grad": Tgrad,  "T_spec": Tspec
})
csv_path = os.path.join(OUTDIR, "ch_triage_timeseries.csv")
df.to_csv(csv_path, index=False)
print("\nSaved:", csv_path)



== CH slope triage (target ~ -1) ==
T_grad vs a_corr : -6.530
T_spec vs a_corr : -3.602
T_grad vs a_peak : -1.814
T_grad vs a_len  : -0.246
T_spec vs a_peak : -0.768

Saved: C:\Users\caleb\CNT_Lab\notebooks\archive\cnt_ch_slope_triage\ch_triage_timeseries.csv


C:\Users\caleb\AppData\Local\Temp\ipykernel_47300\4240350048.py:91: RankWarning: Polyfit may be poorly conditioned
  s,_ = np.polyfit(loga, logT, 1) if loga.size>=12 else (np.nan, np.nan)
C:\Users\caleb\AppData\Local\Temp\ipykernel_47300\4240350048.py:91: RankWarning: Polyfit may be poorly conditioned
  s,_ = np.polyfit(loga, logT, 1) if loga.size>=12 else (np.nan, np.nan)
C:\Users\caleb\AppData\Local\Temp\ipykernel_47300\4240350048.py:91: RankWarning: Polyfit may be poorly conditioned
  s,_ = np.polyfit(loga, logT, 1) if loga.size>=12 else (np.nan, np.nan)


In [11]:
# Stable CH (spectral semi-implicit) + Physics-aligned slope test
# Target: slope ≈ -1 for log Energy vs log a_tv (a_tv = TV0/TV, TV=⟨|∇ψ|⟩)
import os, math, numpy as np, pandas as pd, matplotlib.pyplot as plt
np.seterr(all='ignore'); np.random.seed(7)

OUTDIR = os.path.abspath("./cnt_ch_energy_gate"); os.makedirs(OUTDIR, exist_ok=True)

# --- CH params (robust defaults) ---
N, L = 256, 256.0
DX = L/N
DT = 0.08          # 0.06–0.10 also ok
STEPS = 2400       # longer run -> better dynamic range
SAVE_EVERY = 999999
KAPPA, BETA, MU = 1.0, 1.0, 1.0   # f'(ψ)=βψ^3 - μψ
NOISE_AMP = 8e-4   # tiny symmetry-breaking noise

# --- Spectral grids/operators ---
kx = 2*np.pi*np.fft.fftfreq(N, d=DX)
ky = 2*np.pi*np.fft.fftfreq(N, d=DX)
KX, KY = np.meshgrid(kx, ky, indexing='xy')
K2 = KX**2 + KY**2
DEN = 1.0 + DT * (K2) * (KAPPA*K2 + MU)   # semi-implicit denominator

# 2/3 dealiasing
kx_cut = (2/3)*np.max(np.abs(kx)); ky_cut = (2/3)*np.max(np.abs(ky))
DEAL = (np.abs(KX) <= kx_cut) & (np.abs(KY) <= ky_cut)

# --- Helpers ---
def grad(P):
    gx = (np.roll(P,-1,1) - np.roll(P,1,1))/(2*DX)
    gy = (np.roll(P,-1,0) - np.roll(P,1,0))/(2*DX)
    return gx, gy

def grad_energy(P):
    gx, gy = grad(P)
    return float(np.mean(gx*gx + gy*gy))  # ⟨|∇ψ|²⟩

def total_variation(P):
    gx, gy = grad(P)
    return float(np.mean(np.sqrt(gx*gx + gy*gy) + 1e-12))  # ⟨|∇ψ|⟩

def energy_density(P):
    gx, gy = grad(P)
    W = (BETA/4.0)*(P**4) - (MU/2.0)*(P**2)
    return float(np.mean( (KAPPA/2.0)*(gx*gx + gy*gy) + W ))

def spectral_power(P):
    F = np.fft.fftn(P)
    S = (F*np.conj(F)).real
    return np.fft.fftshift(S)

def radialize(A):
    ny,nx = A.shape; cy,cx = ny//2, nx//2
    y,x = np.indices((ny,nx))
    r = np.sqrt((y-cy)**2 + (x-cx)**2).astype(np.int32)
    rmax = min(cy,cx)
    sums = np.bincount(r.ravel(), weights=A.ravel(), minlength=rmax)
    cnts = np.bincount(r.ravel(), minlength=rmax)
    return np.where(cnts>0, sums/cnts, 0.0)

def corr_length(P):
    F = np.fft.fftn(P); S = (F*np.conj(F)).real
    C = np.fft.ifftn(S).real / P.size
    C = np.fft.fftshift(C)
    Cr = radialize(C)
    if Cr[0] <= 0: return 1.0
    Cn = Cr/Cr[0]; rr = np.arange(len(Cn))*DX
    i0 = max(2, len(rr)//200); i1 = max(i0+10, len(rr)//3)
    y = np.clip(Cn[i0:i1], 1e-12, 1); x = rr[i0:i1]
    s,_ = np.polyfit(x, np.log(y), 1)
    return float(-1.0/s) if s < 0 else 1.0

def robust_slope(a, T, lo=0.25, hi=0.85):
    a = np.asarray(a); T = np.asarray(T)
    m = (a>0) & (T>0)
    a = a[m]; T = T[m]
    if a.size < 60: return np.nan
    idx = np.argsort(a); a = a[idx]; T = T[idx]
    L = len(a); i0, i1 = int(lo*L), max(int(hi*L), int(lo*L)+14)
    loga = np.log(a[i0:i1]); logT = np.log(T[i0:i1])
    s,_ = np.polyfit(loga, logT, 1) if loga.size>=14 else (np.nan, np.nan)
    return float(s)

# --- Initialize ψ with mid/high-k texture, modest amplitude ---
Psi = np.random.normal(0, 1.0, (N,N))
F = np.fft.fftn(Psi)
K2max = np.max(K2) + 1e-12
mask = np.exp(-K2/(0.10*K2max))
F *= (1.0 - 0.6*mask)
F *= DEAL
Psi = np.fft.ifftn(F).real
Psi *= 1.2
Psi += np.random.normal(0, NOISE_AMP, size=Psi.shape)

# --- Anchors ---
TV0 = total_variation(Psi)
xi0 = corr_length(Psi)

a_tv, a_corr = [1.0], [1.0]
E_list, Tgrad_list = [energy_density(Psi)], [grad_energy(Psi)]
t_list = [0.0]

# --- Evolve (semi-implicit, dealiased) ---
for step in range(1, STEPS+1):
    Psi3 = Psi*Psi*Psi
    F_Psi3 = np.fft.fftn(Psi3); F_Psi3 *= DEAL
    RHS = np.fft.fftn(Psi) + DT * (-(K2)) * (BETA * F_Psi3)
    F_next = RHS / DEN
    F_next *= DEAL
    Psi = np.fft.ifftn(F_next).real
    Psi = np.clip(Psi, -4.0, 4.0)
    if NOISE_AMP>0 and step % 50 == 0:
        Psi += np.random.normal(0, NOISE_AMP, size=Psi.shape)

    if step % 5 == 0:
        tv = total_variation(Psi)
        xi = corr_length(Psi)
        a_tv.append( (TV0 + 1e-12)/(tv + 1e-12) )
        a_corr.append( xi/(xi0 + 1e-12) )
        E_list.append( energy_density(Psi) )
        Tgrad_list.append( grad_energy(Psi) )
        t_list.append(step*DT)

# enforce weak monotonicity on the two scale factors
a_tv  = np.maximum.accumulate(np.array(a_tv))
a_corr= np.maximum.accumulate(np.array(a_corr))
E_arr = np.array(E_list)
T_arr = np.array(Tgrad_list)

# --- Slopes ---
s_E_atv   = robust_slope(a_tv,   E_arr)
s_T_atv   = robust_slope(a_tv,   T_arr)
s_E_acorr = robust_slope(a_corr, E_arr)
s_T_acorr = robust_slope(a_corr, T_arr)

print("\n== CH (energy gate) — target slope ≈ -1 ==")
print(f"log Energy vs log a_tv   : {s_E_atv:.3f}")
print(f"log T_grad vs log a_tv   : {s_T_atv:.3f}")
print(f"log Energy vs log a_corr : {s_E_acorr:.3f}")
print(f"log T_grad vs log a_corr : {s_T_acorr:.3f}")

# --- Save series & quick plots ---
df = pd.DataFrame({"t":t_list, "a_tv":a_tv, "a_corr":a_corr, "Energy":E_arr, "T_grad":T_arr})
csv_path = os.path.join(OUTDIR, "ch_energy_gate_timeseries.csv")
df.to_csv(csv_path, index=False)

plt.figure(figsize=(7,4)); plt.plot(t_list, a_tv, linewidth=2)
plt.xlabel("time"); plt.ylabel("a_tv = TV0/TV"); plt.title("CH Expansion (interface scale) a_tv(t)")
plt.tight_layout(); plt.savefig(os.path.join(OUTDIR, "a_tv_vs_time.png"), dpi=160); plt.close()

plt.figure(figsize=(7,4)); plt.plot(t_list, E_arr, linewidth=2)
plt.xlabel("time"); plt.ylabel("Energy density"); plt.title("CH Cooling: Energy(t)")
plt.tight_layout(); plt.savefig(os.path.join(OUTDIR, "energy_vs_time.png"), dpi=160); plt.close()

# Log–log for the target pair
mask = (a_tv>0) & (E_arr>0)
a_use = a_tv[mask]; E_use = E_arr[mask]
idx = np.argsort(a_use); a_use = a_use[idx]; E_use = E_use[idx]
Lw=len(a_use); i0,i1 = int(0.25*Lw), max(int(0.85*Lw), int(0.25*Lw)+14)
loga = np.log(a_use[i0:i1]); logE = np.log(E_use[i0:i1])
m,b = np.polyfit(loga, logE, 1) if loga.size>=14 else (np.nan, np.nan)
plt.figure(figsize=(6,5))
plt.scatter(loga, logE, s=10)
xline = np.linspace(loga.min(), loga.max(), 200)
plt.plot(xline, m*xline + b, linewidth=2)
plt.xlabel("log a_tv"); plt.ylabel("log Energy"); plt.title(f"CH: log Energy vs log a_tv (slope ≈ {m:.3f})")
plt.tight_layout(); plt.savefig(os.path.join(OUTDIR, "loglog_E_vs_atv.png"), dpi=160); plt.close()

print("\nSaved:", OUTDIR)
print("Timeseries:", csv_path)


C:\Users\caleb\AppData\Local\Temp\ipykernel_47300\732409834.py:81: RankWarning: Polyfit may be poorly conditioned
  s,_ = np.polyfit(loga, logT, 1) if loga.size>=14 else (np.nan, np.nan)



== CH (energy gate) — target slope ≈ -1 ==
log Energy vs log a_tv   : nan
log T_grad vs log a_tv   : -1.891
log Energy vs log a_corr : nan
log T_grad vs log a_corr : -6.734

Saved: C:\Users\caleb\CNT_Lab\notebooks\archive\cnt_ch_energy_gate
Timeseries: C:\Users\caleb\CNT_Lab\notebooks\archive\cnt_ch_energy_gate\ch_energy_gate_timeseries.csv
